In [1]:
library(miloR)
library(SingleCellExperiment)
library(scater)
library(dplyr)
library(patchwork)

Loading required package: edgeR

Loading required package: limma

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedi

In [2]:
obj = qs::qread('../00_preprocessing/_targets/objects/exp_labelled_other')
obj

Loading required package: SeuratObject

Attaching sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays


Loading required package: Seurat


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays




An object of class Seurat 
56583 features across 37179 samples within 3 assays 
Active assay: SCT (24262 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [3]:
obj <- obj[, sample(colnames(obj), size = 10000, replace=F)]

In [4]:
obj@meta.data = obj@meta.data %>% mutate(batch = stringr::str_replace_all(batch, stringr::fixed(" "), '__'))

In [5]:
obj@meta.data$time = forcats::fct_recode(obj@meta.data$time, "T1"="Day5", "T2"="Day14")

In [6]:
obj@meta.data$treatment = forcats::fct_recode(obj@meta.data$treatment, "P0"="Veh_PF", "P1"="FGF1")

In [7]:
obj@meta.data$strain = forcats::fct_recode(obj@meta.data$strain, "S0"="BL6", "S1"="obob")

In [8]:
levels(obj@meta.data$strain)

[1] "S0" "S1"

In [9]:
clusters = as.list(paste0("c", seq(1:length(levels(obj@meta.data$labels)))))
names(clusters) = levels(obj@meta.data$labels)
clusters

$Astrocytes
[1] "c1"

$`a1_Tany-Astrocytes`
[1] "c2"

$MOL
[1] "c3"

$a2_Tanycytes
[1] "c4"

$NG2_OPC2
[1] "c5"

$`a2_Tanycytes-b1_Tanycytes`
[1] "c6"

$b1_Tanycytes
[1] "c7"

$b2_Tany
[1] "c8"

$Microglia
[1] "c9"

$Epend
[1] "c10"

$`NG2_OPC2-Microglia`
[1] "c11"

$NFOL
[1] "c12"

$Mural_Cells2
[1] "c13"

$`Fibroblasts2-Fibroblasts3`
[1] "c14"

$`Parstuber2A-ParsTuber1`
[1] "c15"

$Endothelial
[1] "c16"

$`MOL-a2_Tanycytes`
[1] "c17"

In [10]:
obj@meta.data$clusters = obj@meta.data$labels
levels(obj@meta.data$clusters) = paste0("c", seq(1:length(levels(obj@meta.data$labels))))
head(obj@meta.data$clusters)
head(obj@meta.data$labels)

[1] c6  c1  c12 c2  c1  c10
Levels: c1 c2 c3 c4 c5 c6 c7 c8 c9 c10 c11 c12 c13 c14 c15 c16 c17

[1] a2_Tanycytes-b1_Tanycytes Astrocytes               
[3] NFOL                      a1_Tany-Astrocytes       
[5] Astrocytes                Epend                    
17 Levels: Astrocytes a1_Tany-Astrocytes MOL a2_Tanycytes ... MOL-a2_Tanycytes

In [11]:
obj@meta.data$group = interaction(obj@meta.data$treatment, obj@meta.data$time, obj@meta.data$strain, drop = TRUE)

In [12]:
colnames(obj@meta.data)

[1] "orig.ident"                    "nCount_RNA"                   
 [3] "nFeature_RNA"                  "Index.10x"                    
 [5] "nCount_HTO"                    "nFeature_HTO"                 
 [7] "Index.HTO"                     "nCount_spliced"               
 [9] "nFeature_spliced"              "nCount_unspliced"             
[11] "nFeature_unspliced"            "HTO_mcl_maxID"                
[13] "HTO_mcl_secondID"              "HTO_mcl_margin"               
[15] "HTO_mcl_classification"        "HTO_mcl_classification.global"
[17] "hash.mcl.ID"                   "doublet"                      
[19] "predicted_dub_std"             "predicted_dub_cut"            
[21] "predicted_dub_all"             "nCount_SCT"                   
[23] "nFeature_SCT"                  "SCT_snn_res.0.8"              
[25] "seurat_clusters"               "strain"                       
[27] "treatment"                     "time"                         
[29] "collection_date"               "isolation_date"               
[31] "batch"                         "predicted.id"                 
[33] "prediction.score.max"          "labels"                       
[35] "clusters"                      "group"

In [13]:
obj@meta.data = obj@meta.data %>%
select(c("hash.mcl.ID", "batch", "group", "clusters"))

In [14]:
source('../00_preprocessing/preprocessing.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.7          ✔ purrr   0.3.4     
✔ tidyr   1.2.0          ✔ stringr 1.4.0     
✔ readr   2.1.2          ✔ forcats 0.5.2.9000

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()      masks matrixStats::count()
✖ dplyr::desc()       masks IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::lag()        masks stats::lag()
✖ ggplot2::Position() masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()     masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()     masks S4Vectors::rename()
✖ dplyr::slice()      masks IRanges::slice()



In [15]:
obj = Seurat::SCTransform(obj,
                        assay='RNA',
                        method="glmGamPoi",
                        vars.to.regress="batch",
                        vst.flavor="v2",
                        verbose=TRUE)
obj = run_sct_chaser(obj, resolution=0.8)
obj

vst.flavor='v2' set, setting model to use fixed slope and exclude poisson genes.

Calculating cell attributes from input UMI matrix: log_umi

Total Step 1 genes: 20273

Total overdispersed genes: 18494

Excluding 1779 genes from Step 1 because they are not overdispersed.

Variance stabilizing transformation of count matrix of size 21813 by 10000

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |======================================================================| 100%


Setting estimate of  103 genes to inf as theta_mm/theta_mle < 1e-3

# of step1 poisson genes (variance < mean): 0

# of low mean genes (mean < 0.001): 1530

Total # of Step1 poisson genes (theta=Inf; variance < mean): 121

Total # of poisson genes (theta=Inf; variance < mean): 3215

Calling offset model for all 3215 poisson genes

Found 115 outliers - those will be ignored in fitting/regularization step


Ignoring theta inf genes

Replacing fit params for 3215 poisson genes by theta=Inf

Setting min_variance based on median UMI:  0.04

Second step: Get residuals using fitted parameters for 21813 genes



  |======================================================================| 100%


Computing corrected count matrix for 21813 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 3.825253 mins

Determine variable features

Place corrected count matrix in counts slot

Regressing out batch

Centering data matrix

Set default assay to SCT

Warning message:
"Invalid name supplied, making object name syntactically valid. New object name is Seurat..SCTransform.RNA; see ?make.names for more details on syntax validity"
Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"
19:42:34 UMAP embedding parameters a = 0.9922 b = 1.112

19:42:34 Read 10000 rows and found 30 numeric columns

19:42:34 Using Annoy for neighbor search, n_neighbors = 30

19:42:34 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----

An object of class Seurat 
54134 features across 10000 samples within 3 assays 
Active assay: SCT (21813 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [16]:
obj@meta.data

,hash.mcl.ID,batch,group,clusters,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters
,<chr>,<chr>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
GATGCTAAGTCACGAG_1_1,M17,Batch__1,P0.T2.S0,c6,7981,3207,7,7
GTACAGTGTCGGTGAA_3_2,M25,Batch__2,P0.T2.S1,c1,7825,2816,1,1
GGTAACTTCCATAGGT_2_3,N21,Batch__3,P1.T1.S1,c12,7533,2479,14,14
ACGGAAGAGTGCGTCC_5_1,M24,Batch__1,P0.T2.S1,c2,6151,1529,4,4
CTAGACAAGCTGTTAC_2_2,M11,Batch__2,P0.T1.S1,c1,7935,2861,1,1
CTTACCGGTACCGGCT_5_1,M18,Batch__1,P0.T2.S0,c10,6590,2279,12,12
CTCAGGGAGCCGAACA_5_1,M24,Batch__1,P0.T2.S1,c5,7375,2610,5,5
ACTTTGTTCTTAAGGC_2_2,M19,Batch__2,P0.T2.S0,c1,6778,1927,0,0
TTCTGTACAAATGATG_3_3,N23,Batch__3,P0.T1.S1,c2,7178,2486,1,1


In [17]:
saveRDS(obj, 'ds_obj.rds')

In [18]:
qs::qsave(obj, 'ds_obj.qs')

In [19]:
2

[1] 2